In [2]:
from bonndit.michi import dwmri
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, reorient_bvecs
import nibabel as nib
import numpy as np

/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from subprocess import check_output
from os.path import expanduser
import os
home = expanduser("~")
DATA_DIR = os.path.join(home, 'Devel/test/samuel/in4michi')

In [5]:
data, dwmri_gtab, meta = dwmri.load(os.path.join(DATA_DIR, 'data.nii.gz'))

data = nib.load(os.path.join(DATA_DIR, "data.nii.gz"))
bvals, bvecs = read_bvals_bvecs(os.path.join(DATA_DIR, 'bvals'), os.path.join(DATA_DIR, 'bvecs'))
gtab = gradient_table(bvals, bvecs)
dipy_gtab = reorient_bvecs(gtab, np.array([data.affine]*len(gtab.bvals[gtab.bvals!=0])))



In [38]:
def fsl_to_worldspace_old(data, gtab):
    # we have to bring b vectors into world coordinate system
    # we will use the 3x3 linear transformation part of the affine matrix for this
    linear = data.affine[0:3, 0:3]
    # according to FSL documentation, we first have to flip the sign of the
    # x coordinate if the matrix determinant is positive
    bvecs = gtab.bvecs
    if np.linalg.det(linear) > 0:
        bvecs[:, 0] = -bvecs[:, 0]
    # now, apply the linear mapping to bvecs and re-normalize
    bvecs = np.dot(bvecs, np.transpose(linear))
    bvecnorm = np.linalg.norm(bvecs, axis=1)
    bvecnorm[bvecnorm == 0] = 1.0  # avoid division by zero
    bvecs = bvecs / bvecnorm[:, None]
    return gradient_table(gtab.bvals, bvecs)

def fsl_to_worldspace_new(data, gtab):
    from scipy.linalg import polar
    # we have to bring b vectors into world coordinate system
    # we will use the 3x3 linear transformation part of the affine matrix for this
    linear = data.affine[0:3, 0:3]
    # according to FSL documentation, we first have to flip the sign of the
    # x coordinate if the matrix determinant is positive
    bvecs = gtab.bvecs
    if np.linalg.det(linear) > 0:
        bvecs[:, 0] = -bvecs[:, 0]
    
    rotation, scaling = polar(linear)
    bvecs = np.dot(bvecs, np.transpose(rotation))
    return gradient_table(gtab.bvals, bvecs)

In [56]:
%%timeit -n 1000 -r 5
old = fsl_to_worldspace_old(data, gtab)

141 µs ± 7.89 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [57]:
%%timeit -n 1000 -r 5
new = fsl_to_worldspace_new(data, gtab)

230 µs ± 9.31 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [41]:
diff = (fsl_to_worldspace_old(data, gtab).bvecs - fsl_to_worldspace_new(data, gtab).bvecs)/fsl_to_worldspace_old(data, gtab).bvecs

/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [55]:
max(diff.flatten()[~np.isnan(diff.flatten())])

0.00020034853598404598

In [ ]:
from bonndit.michi.storage import affine_to_meta3d

affine =np.array([[1, 0, 1,0],
                  [0, 1, 0,0],
                  [0, 0, 0,0],
                  [0, 0, 0,1]])

meta = affine_to_meta3d(affine)
bvecs = np.array([[1, 0, 0],
                 [3/5, 4/5, 0],
                 [0, 0, 1]])

bvals = np.array([700]*3)
gtab = gradient_table(bvals, bvecs)
reorient_bvecs(gtab, np.array([affine]*3)).bvecs

In [ ]:
dwmri._nifti_bvecs_to_worldspace(bvecs, meta)